# Introduction and Relational Databases

## Database Introduction

### 7 Aspects of Database Systems

- massive: terabytes.
- persistent: the data is what sits there and then program will start up, it will operate on the data, the program will stop and the data will still be there. 
- safe: hardware, software, power, users.
- multi-user: concurrency control.
- convenient: 
    - physical data independence, data is actually stored and laid out on disk is independent of the way that programs think about the structure of the data.
    - high-level query language, declarative
- efficient: thousands of queries/updates persecond.
- reliable: 99.99999% up time.

### A Few of Aspects Surrounding Database Systems

- Database applications may be programmed via "framework".
- DBMS may run in conjunction with "middleware".
- Data-intensive application may not use DBMS at all. e.g.: Hahoop is a processing framework for running operations on data that's stored in files.

### Key Concepts

- data model: 
    - set of records. 
    - XML documents: an XML document captures data, instead of a set of records, as a hierarchical structure, of labeled values; 
    - graph data model: all data in the database is in the form of nodes and edges.
- schema vs data: like types and variables in program language.
- data definition data (DDL): to set up schema.
- data manipulation/query language (DML): to query and modify.

### Key People

- DBMS implementer: build system.
- database designer: establish schema.
- database application developer: programs that operate on database. It's not necessary to have a one-to-one coupling between database and programs. For example, have a sales database where some applications are actually inserting the sales as they happen, while others are analyzing the sales.
- database administrator: loads data, keep running smoothly, tuning.

## Relational Database

- used by all major commercial databse systems
- very simple model
- query with high-level languages: simple yet expressive
- efficient implementations.

### Key Concepts

- database = set of `relations` (or `tables`). table name can be singular or palural, keep consistent.
- each relation has a set of named `attributes` (or `columns`).
- each `tuple` (or `row`) has a value for each attribute.
- each attribute has a `type` (or `domain`), atomic type or structured type.
- `schema`: structural description of relations in database.
- `instance`: actual contents at given point of time.
- `NULL`: special value for 'unknown' or 'undefined'.
- `key`: attribute whose value is unique in each tuple, or set of attributes whose combined values are unique. 
    - database systems for efficiency tend to build special index structures or store the database in a particular way.
    - if one relation in a relational database wants to refer to tuples of another, there 's no concept of pointer in relational databases. Therefore, the first relation will typically refer to a tuple in the second relation by its unique key.
    
```sql
/* creating table example */

Create Table Student
    (ID, name, GPA, photo)

Create Table College
    (name string, state char(2), enrollment integer)
```

## Querying Relational Database

### Steps in Creating and Using a Relational Database

1. Design schema, create using DDL;
2. "Bulk load" initial data;
3. Repeat and can be used by muti-users: execute queries and modifications;

### Ad-hoc Queries in High-level Languages

- Ad-hoc: you can pose queries that you didn't think of in advance.
- High-level: you can write in a fairly compact fashion rather complicated queries and you don't have to write the algorithms that get the data out of the database.
- Sample query questions: 
    - some are easy to pose, some are harder;
    - some query are more efficent, some are harder;
    - "Query Language" (DML) is not only for query but also for modify;

```sql
/* all students with GPA > 3.7 applying to Stanford and MIT only */

/* all engineering departments in CA with < 500 applicants */

/* college with highest average accept rate over last 5 years */
```

### Queries Return Relations

- compositional;
- closed: get back the same type of object that you query;

### Query Languages

- Relational Algebra: formal;
- SQL: actual/implemented, does have as its foundation relational algebra;

```sql
/* relational algebra */

Π ID (σ GPA>3.7 ^ cName='Stanford' (Student ⋈ Apply))

/* SQL */

SELECT Student.ID FROM Student, Apply
WHERE Student.ID=Apply.ID AND GPA>3.7 AND cName='Stanford'
```

# Relational Algebra

## Select, Project, Join

```sql
/* Example in this section: */
College (cName, state, enrollment)
Student (sID, sName, GPA, sizeHS)
Apply (sID, cName, major, decision)
```

- __select__: `σ`, pick certain rows;

```sql
/* students with GPA>3.7 */
σ GPA>3.7 Student

/* students with GPA>3.7 and HS<1000 */
σ GPA>3.7 ^ sizeHS<1000 Student

/* Aplications to Stanford CS major */
σ cName='Stanford' ^ major='CS' Apply
```

- __project__: `Π`, pick certain columns;

```sql
/* ID and dicision of all applicants */
Π ID,decision Apply

/* ID and sName of students with GPA>3.7 */
Π ID,decision (σ GPA>3.7 Student)
-- compose selection and projection is useful, but compose two selection or two projection is not good practice.
```

- Relational Algebra treat duplicates not like SQL:
    - SQL: multisets/bags, which means duplicates will not be eleminated;
    - Relational Algegra: sets, which means duplicates will be eleminated;
    
```sql
/* Dealing with duplicates */

/* following projection will have duplicates in SQL */
/* list of application majors and decisions */
Π maijor, decision Apply
-- no duplicates in result
```

- __cross join__: `×` (cross-product, or Certesian Product, [笛卡尔积](https://zh.wikipedia.org/wiki/%E7%AC%9B%E5%8D%A1%E5%84%BF%E7%A7%AF))

```sql
/* Names and GPAs of students with sizeHS>1000 who applied to CS and were rejected */

/* all the following expressions are correct */
Π name,GPA (σ Student.sID=Apply.sID(Student × Apply) ^ sizeHS>1000 ^ major='CS' ^ decision='reject')

Πname,GPA(σ Student.sID=Apply.sID^sizeHS>1000^major='CS'^decision='reject'(Student×ΠsID,major,decision Apply))

Π name,GPA (σ Student.sID=Apply.sID ((σ sizeHS>1000 Student) × (σ major='CS' ^ decision='reject' Apply)))
```

- __natural join__: `⋈`
    - enforce equality on all attributes with same name automatically;
    - eliminate one copy of duplicate attributes;


```sql
/* the following expression is true */
Exp1 ⋈ Exp2 === Π schema(E1) U schema(E2) (σ E1.A1=E2.A1 ^ E1.A2=E2.A2 ^ ... (Exp1 × Exp2))
```

```sql
/* Names and GPAs of students with sizeHS>1000 who applied to CS and were rejected */
Π sName,GPA (σ sizeHS>1000 ^ major='CS' ^ decision='reject' (Student ⋈ Apply))

/* Names and GPAs of students with sizeHS>1000 who applied to CS and were rejected at college with enroment>2000 */
Π sName,GPA (σ sizeHS>1000 ^ major='CS' ^ decision='reject' ^ enroment>2000 (Student ⋈ (Apply ⋈ College)))
-- College and Apply should be joined first by cName or Student and Apply should be joined first by sID
```


- __theta join__: `⋈θ`
    - Basic operation implemented in DBMS;
    - Term `join` often means theta join;

```sql
/* the following expression is true */
Exp1 ⋈θ Exp2 === σ θ (Exp1 × Exp2)
-- where θ is the condition
```

## Set Operators

- __Union__: `∪`

```sql
/* list of college and student names */

Π cName College ∪ Π sName Student
```

- __Difference__: `-`

```sql
/* IDs of students who didn't apply any where */
Π sID Student - Π sID Apply

/* Names of students who didn't apply any where */
Π sName((Π sID Student - Π sID Apply) ⋈ Student)
```

- __Intersections__: `∩`

```sql
/* Names that are both a college name and a student name */
Π sName Student ∩ Π cName College

/* intersection doesn't add any new express power */
E1 ∩ E2 = E1 - (E1 - E2)

/* if E1 schema === E2 schema, then: */
E1 ∩ E2 = E1 ⋈ E2
```

## Renaming

- To unify schemas for set operator

```sql
/* list of college and student names */
ρ c(name) (Π cName College) ∪ ρ c(name) (Π sName Student)
-- rename and then union
```

- For disambiguation in "self-joins"

```sql
/* pairs of colleges in same state */
σ state=state (College × College) -- wrong way, we could not have done this query without the rename operator.
σ s1=s2((ρ c1(n1, s1, e1)(College)) × (ρ c2(n2, s2, e2)(College))) -- right way, rename and then select.
σ n1<n2(ρ c1(n1, s, e1)(College)) ⋈ (ρ c2(n2, s, e2)(College)) -- another way, n1<n2 make sure no duplication
```


## Alternate Notation

- __Assignment__: `:=`

```sql
/* pair of college in same state */
C1 := ρ c1,s,e1 College
C2 := ρ c2,s,e2 College
Cp := C1 ⋈ C2
Ans := σ n1<n2 Cp
```

- __Expression Tree__:

```sql
/* GPAs of students applying to CS in CA */
Π GPA
|
σ state='CA' ^ major='CS'
|
⋈_____
|  |  |
C, S, A
```

## Relational Algebra Summary

### Core Relation

- `R`: relation name
- `σc(E)`: select operator applies a condition to the result of an expression.
- `Πa1,a2...an(E)`: project operator that gives us a set of attributes that we take from the result of an expression.
- `E1 × E2`: expression one cross-product expression two.
- `E1 ∪ E2`: expression one union expression two.
- `E1 - E2`: expression one minus expression two.
- `ρR(a1,a2...an)(E)`: rename operator that takes an expression and renames the result (to `R(a1,a2...an)`) of that, the schema in the result of that expression.

### Abbrev

- `E1 ⋈ E2`
- `E1 ⋈θ E2`
- `E1 ∩ E2`

### Conclusion

- It's a formal language.
- It's based on sets, set operators and other operators that combine data from multiple relations.
- It takes relations as input, it produces relations as answers.
- It does form the formal foundation of implemented relational database management.

# Relational Design Theory

## Overview

- Design by decomposition:
    - start with "meta" relations containing everything;
    - decompose into smaller, better relations with same info;
    - can do decomposition automatically;
    
    
- Automatic decomposition:
    - "Mega" relations + properties of the data;
    - system decomposes based on properties;
    - final set of relations satisfies normal form (no anomalize, no lost of information)
    
    
__Properties and Normal Forms__

- Functional dependencies => Boyce-Codd Normal Form (most commonly used);

    ```sql
    R(ID, A1, A2)
    
    Apply(sID, sName, cName) 
    -- a student can only have one name at a time, but can apply to multi College
    -- sID -> sName, functional dependency
    -- BUT not the case to cName
    
    /* decompose */
    Student(sID, sName) -- BCNF
    Apply(sID, cName)
    ```
    
    - Functional dependencies: ID -> A1, same ID always has same A1, should store each ID's A1 only once;
    - Boyce-Codd Normal Form: for all atttibutes, should be: ID -> A1, ID-> A2, ..., then ID is a key;


- Multivalued dependences => Forth Normal Form (is stricter than Boyce-Codd);

    ```sql
    R(ID, A1, A2)
    
    Apply(sID, cName, HS)
    -- a student can be moved among a lot of High Schools, as well as apply to multi College
    -- sID ->-> cName, sID ->-> HS, multivalue dependencies
    -- BUT given sID has every combination of cName and HS
    
    /* decompose */
    HighSchoole(sID, HS) -- 4NF
    Apply(sID, cName) -- 4NF
    ```
    
    - Forth Normal Form: ID ->-> A1, ID is a key.
    - What happend to first, second and third?
        - first normal form is pretty specifying that relations are real relations with atomic values in each cell, is not discussed very much anymore.
        - second normal form is specifying something about the way relations are structured with respect to their keys, is not discussed very much anymore.
        - third normal form is a slight weakening of Boyce-Codd normal form and sometimes people do like to talk about third normal form.

## Functional Dependencies

```sql
/* Example in this section: */
Student (SSN, sName, address, HScode, HSname, HScity, GPA, priority)
Apply (SSN, cName, state, data, major)
/* support priority is determined by GPA:
    3.8 < GPA       : priority = 1
    3.3 < GPA <= 3.8: priority = 2
          GPA <  3.3: priority = 3
*/
```

```sql
/* In relation Student, we have function dependencies, such as: */

SSN -> sName, address, priority

HScode -> HSname, HScity
HSname, HScity -> HScode

GPA -> priority


/* In relation Apply, we can define some constraints, and then have function dependencies: */
cName -> date        -- if the college with same name can only be applied on the same day
SSN, cName -> major  -- if a student can only apply one major in the college with the same name
SSN -> state         -- if a student can only apply colleges in the same state
```

### Functional Dependencies (FD) and Keys

```sql
/* suppose we have a relation: */
R(A1, A2, ..., An, B1, B2, ...Bm, C1, C2, ..., Ci)
-- Relation with no duplicates;
-- Suppose: {A1, A2, ...,An} -> {B1, B2, ...Bm};
```

- Trival FD:
    - if `{A1, A2, ...} -> {B1, B2, ...}` and `{B1, B2, ...} isSubsetOf {A1, A2, ...}`


- Nontrival FD
    - if `{A1, A2, ...} -> {B1, B2, ...}` and `{B1, B2, ...} not isSubsetOf {A1, A2, ...}`


- Completely Nontrival FD
    - if `{A1, A2, ...} -> {B1, B2, ...}` and `{A1, A2, ...} ∩ {B1, B2, ...} = {}`
    
    
### Rules for FD

- Spliting rule:
    - `{A1, A2, ...} -> {B1, B2, ...}` `=>` `{A1, A2, ...} -> B1`, `{A1, A2, ...} -> B2`, ...;


- Combining rule:
    - `{A1, A2, ...} -> B1`, `{A1, A2, ...} -> B2`, ... `=>` `{A1, A2, ...} -> {B1, B2, ...}`;
    
    
- Trivial-dependency rules:
    - if `{A1, A2, ...} -> {B1, B2, ...}` and `{B1, B2, ...} isSubsetOf {A1, A2, ...}`, then:
        - `{A1, A2, ...} -> {A1, A2, ...} ∪ {B1, B2, ...}`
        - `{A1, A2, ...} -> {A1, A2, ...} ∩ {B1, B2, ...}`
        
       
- Transtive rule:
    - if `{A1, A2, ...} -> {B1, B2, ...}` and `{B1, B2, ...} -> {C1, C2, ...}`, then:
        - `{A1, A2, ...} -> {C1, C2, ...}`
        
        
### Closure of Attributes

- What does closure mean: 
    - given Relations, FDs, and `{A1, A2, ...}`, find all `B` that `{A1, A2, ...} -> B`;
    - reffered to as `{A1, A2, ...}+`


- Closure and keys:
    - if `{A1, A2, ...}+ = allAttributes`, `{A1, A2, ...}` is a key, and the superset of `{A1, A2, ...}` is also a key;


```sql
/* Example */
Student (SSN, sName, address, HScode, HSname, HScity, GPA, priority)

SSN -> sName, address, GPA
GPA -> priority
HScode -> HSname, HScity

{SSN, HScode}+ = {SSN, sName, address, HScode, HSname, HScity, GPA, priority}
-- SSN & HScode ditermin all attributes in Student relation, so SSN & HScode together form a key for the relation
```


### Specifying FDs for a Relation

- `S1` and `S2` are sets of FDs, `S2` "follows from" `S1` if every relation instance satisfying `S1` also satisfies `S2`

```sql
S2: {SSN -> priority}
S1: {SSN -> GPA, GPA -> priority}
```

- How to test `{A1, A2, ...} -> {B1, B2, ...}` follow from `S`
    - method 1: `{A1, A2, ...}+` based on `S`, check if all `B` in `{B1, B2, ...}` in set;
    - method 2: Armstrong's Axioms
    
    
- What we would like to find is a minimal set of completely non-trivial functional dependencies such that all functional dependencies that hold on the relation follow from the dependencies in this set.

## Boyce-Codd Normal Form (BCNF)

- relation `R` with FDs is in BCNF if: 
    - for each `{A1, A2, ...} -> B`, `{A1, A2, ...}` is a key, which means the relation can not be break down further more. (`{A1, A2, ...}` and `{B1, B2, ...}` belongTo `R`, but may not cover all attributes.)
    
### BCNF Decomposition Algorithm

- Input: relation `R`, `FDs` for R;
- Output: decomposition of R into BCNF relations with "lossless join";


- Algorithm:
    - compute keys for `R` using `FDs`;
    - repeat until all relations are in BCNF:
        - pick any `R'` with `A->B` violates BCNF;
        - decompose `R'` into `R1(A, B)` and `R2(A, rest)`;
        - compute `FDs` for `R1` and `R2`;
        - compute `keys` for `R1` and `R2`;

## Multivalued Dependencies (MVDs)

- Based on knowledge of real world;
- All instances of relation must adhere;

### Trivial and Nontrivial MVD

- If `{A1, A2, ...} ->> {B1, B2, ...}`, `{B1, B2, ...} belongTo {A1, A2, ...}` or `{A1, A2, ...} ∪ {B1, B2, ...} = allAttributes`, then it is trival.
- Otherwise, nontrival.

### Rules for MVDs

- FD is an MVD rule. That means, all the rule apply to MVDs, also apply to FDs.

- Intersection:
    - if `{A1, A2, ...} ->> {B1, B2, ...}` and `{A1, A2, ...} ->> {C1, C2, ...}`, then:
        - `{A1, A2, ...} ->> {B1, B2, ...} ∩ {C1, C2, ...}`
    
- Transtive:
    - if `{A1, A2, ...} ->> {B1, B2, ...}` and `{B1, B2, ...} ->> {C1, C2, ...}`, then:
        - `{A1, A2, ...} ->> {B1, B2, ...} - {C1, C2, ...}`

## Fourth Normal Form (4NF)

- relation `R` with MVDs is in 4NF if: 
    - for each nontrivial `{A1, A2, ...} ->> B`, `{A1, A2, ...}` is a key.
    - 4NF is stronger than BCNF, which means, 4NF is inside the BCNF and implies BCNF.
    
### 4NF Decomposition Algorithm

- Input: relation `R`, `FDs` for R, `MVDs` for R;
- Output: decomposition of R into 4NF relations with "lossless join";


- Algorithm:
    - compute keys for `R` using `FDs`;
    - repeat until all relations are in 4NF:
        - pick any `R'` with `A->>B` violates 4NF;
        - decompose `R'` into `R1(A, B)` and `R2(A, rest)`;
        - compute `FDs` and `MVDs` for `R1` and `R2`;
        - compute `keys` for `R1` and `R2`;

## Shortcomings of BCNF and 4NF

- dependency enforcement
- query workload
- over-decomposition

```sql
/* Example */
Apply(SSN, cName, date, major)
-- can apply to each college once for one major
-- colleges have non-overlapping application dates

-- FDs: 
    -- SSN, cName -> date, major (first consumption)
    -- date -> cName (second consumption)
-- Keys:
    -- SSN, cName
-- BCNF decomposion:
    A1(date, cName)
    A2(SSN, date, major)
    -- but is not a good design
    
-- 3NF is enough: keep what it is is already a 3NF
```

```sql
/* Example */
Student(SSN, HSname, GPA, priority)
-- multiple HSname is allowed
-- GPA determins priority

-- FDs: 
    -- SSN -> GPA, GPA -> priority, SSN -> priority
-- Keys:
    -- SSN, HSname
-- BCNF decomposion:
    S1(SSN, GPA)
    S2(SSN, priority)
    S3(SSN, HSname)
    -- but is not a good design, we lost GPA -> priority dependency, better like this:
    S1(SSN, GPA, priority)
    S2(SSN, HSname)
```

```sql
/* Example */
Scores(SSN, sName, SAT, ACT)
-- multiple SATs and ACTs allowed
-- all queries return name + coposite score

-- FDs: 
    -- SSN -> sName
-- Keys:
    -- no keys
-- 4NF decomposion:
    S1(SSN, sName)
    S2(SSN, SAT)
    S3(SSN, ACT)
    -- it seems a good design, but not for every query want to return name + coposite score
    -- original design is suitable for the second consumption (denormalized relation)
```

## Unified Modeling Language

- Data modeling: how to represent data for application
    - relational model - with design principles
    - XML
    - database design model:
        - not implemented by system;
        - translated from higher-level model into model of DBMS
        
### Higher-Level Database Design Models

- Entity-Relation model (E/R) & Unified Modeling Language (UML):
    - both are graphical;
    - both can be translated to relations (semi-)automatically;
    - UML is widly used not only in database, but also in programming;
    
### 5 Concepts of Unified Modeling Language

- Classes;
- Associations & Association Classes;
- Subclasses;
- Composition and Aggregation;

#### Classes

- name
- attributes
- methods

```
|Student  |
|---------|
|sID, pk  |
|sName    |
|GPA      |
|---------|
|<methods>|

Translation: Student(sID, sName, GPA)
```

```
|College  |
|---------|
|cName, pk|
|state    |
|enrolment|
|---------|
|<methods>|

Translation: College(cName, state, enrolment)
```

#### Associations and Association Classes

- associations: relationships between objects of two classes;
- association classes: relationships between objects of two classes;
- multiplication associations:
    - each object of C1 has at least `m` and most `n` objects of C2;
        - `*` can represent any number of objects.
        - default is `1..1` on both side.
        - `1..1` can abbreviate as `1`.
        - `0..*` can abbreviate as `*`.
    - types:
        - one to one
        - many to one
        - many to many
        - complete


```
|Student  |                  |College  |
|---------|                  |---------|
|sID, pk  | 0..1  Applied *  |cName, pk|
|sName    | ---------------> |state    |
|GPA      |   |AppInfo   |   |enrolment|
|---------|   |----------|   
|<methods>|   |date      |
              |decision  |

Translation:
    Student(sID, ...)
    College(cName, ...)
    AppInfo(sID, cName, ...), cName is the key (new key always in * side)

Canbe implified if Applied is empty except the keys from Student and College:
    Student(sID, ...)
    College(cName, ..., sID) / College(cName, ..., null)
```

- self-associations:

```
|Student  |
|---------|
|sID, pk  |______
|sName    |   *  |
|GPA      |      | 
|---------|      |
|<methods>|      |
                 |Sibling
|Student  |      |
|---------|      |
|sID, pk  |______|
|sName    |   *
|GPA      |
|---------|
|<methods>|

Translation:
    Student(sID, sName, GPA)
    Sibling(sID1, sID2), sID1 and sID2 together is a key
```

```
|College  |
|---------|home
|cName, pk|______
|state    |   1  |
|enrolment|      |
                 |Branch
|College  |      |
|---------|      |
|cName, pk|______|
|state    |   *
|enrolment|satellite


Translation:
    College(cName, state)
    Branch(home, satellite), satellite is the key
```

- eliminating association classes:
    - unnecessary if `0..1`/`1` to `0..1`/`1`;
    - if `*` to `0..1`/`1`, just move association class to the `*` side.


#### Subclasses

- superclass = generalization
- subclass   = specialization
- incomplete (partial) vs complete
- disjoint (exclusiv)  vs overlapping

```
|Student  |
|---------|(complete and overlapping, with ForeignS, DomesticS, and APStudent as subclasses)
|sID, pk  |(it's complete and disjoint, if without APStudent)
|sName    |(it's incomplete if only with APStudent)
   __|____________________________________
   |                  |                  | 
|ForeignS |      |DomesticS |       |APStudent | *       1..10 |APCourse   |
|---------|      |----------|       |----------| ------------> |-----------|
|country  |      |state     |         (empty)      |AppInfo|   |course#, pk|
                 |SSN       |                      |-------|   |title      |
                                                   |year   |
                                                   |grade  |
                                                   
General Translation (3 ways):
1. Student(sID, sName), ForeignS(sID, country), DomesticS(sID, state, SSN), ...
2. Student(sID, sName), ForeignS(sID, sNAme, country), DomesticS(sID, sNAme, state, SSN), ...
3. Student(sID, sName, country, state, SSN)

In heavily overlapping => use 3.
In disjoint and complete => use 2.

In this paticular situation, the best way:
    Student(sID, sName)
    ForeignS(sID, country) -- subclass can translate keys from superclass automatically
    DomesticS(sID, state, SSN)
    APStudent(sID)         -- this relation is redundent, could be eliminated
    APCourse(SID, course#, title)
    AppInfo(sID, course#, year, grade) -- association class can translate keys automatically
```

#### Composition and Aggregation

- objects of one class belong to  objects of another class;

```
|College  |           |Department|(composition)
|---------|  1        |----------|
|cName, pk| <#>-------|dName, pk |     |Apartment |(aggregation)
|state    |           |building  |     |----------|
|---------| < >------------------------|addr, pk  |
|<methods>| 0..1                       |unit#     |

Translation:
    College(cName, state)
    Department(dName, building, cName)
    Apartment(addr, unit, cName/null)
```